# בנה מודל רגרסיה: מודלים של רגרסיה לינארית ופולינומית


## רגרסיה ליניארית ופולינומית לתמחור דלעות - שיעור 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>אינפוגרפיקה מאת דאסאני מדיפאלי</figcaption>

#### הקדמה

עד כה חקרתם מהי רגרסיה באמצעות נתוני דוגמה שנאספו ממאגר הנתונים של תמחור דלעות שבו נשתמש לאורך השיעור הזה. בנוסף, גם ביצעתם ויזואליזציה של הנתונים באמצעות `ggplot2`. 💪

עכשיו אתם מוכנים לצלול עמוק יותר לתוך עולם הרגרסיה עבור למידת מכונה. בשיעור הזה תלמדו יותר על שני סוגי רגרסיה: *רגרסיה ליניארית בסיסית* ו-*רגרסיה פולינומית*, יחד עם חלק מהמתמטיקה שמאחורי הטכניקות הללו.

> לאורך הקורס הזה, אנו מניחים ידע מתמטי מינימלי ושואפים להפוך את החומר לנגיש עבור תלמידים שמגיעים מתחומים אחרים. שימו לב להערות, 🧮 קריאות, דיאגרמות וכלים נוספים שיעזרו בהבנה.

#### הכנה

כתזכורת, אתם טוענים את הנתונים הללו כדי לשאול שאלות לגביהם.

- מתי הזמן הטוב ביותר לקנות דלעות?

- איזה מחיר אני יכול לצפות עבור ארגז של דלעות מיניאטוריות?

- האם כדאי לי לקנות אותן בסלים של חצי בושל או בארגזים של 1 1/9 בושל? בואו נמשיך לחקור את הנתונים הללו.

בשיעור הקודם יצרתם `tibble` (גרסה מודרנית של מסגרת נתונים) ומילאתם אותו עם חלק ממאגר הנתונים המקורי, תוך סטנדרטיזציה של התמחור לפי בושל. עם זאת, על ידי כך הצלחתם לאסוף רק כ-400 נקודות נתונים ורק עבור חודשי הסתיו. אולי נוכל לקבל קצת יותר פרטים על טיב הנתונים על ידי ניקוי נוסף שלהם? נראה... 🕵️‍♀️

למשימה הזו נזדקק לחבילות הבאות:

- `tidyverse`: [tidyverse](https://www.tidyverse.org/) הוא [אוסף של חבילות R](https://www.tidyverse.org/packages) שנועד להפוך את מדע הנתונים למהיר, קל ומהנה יותר!

- `tidymodels`: [tidymodels](https://www.tidymodels.org/) הוא מסגרת [אוסף של חבילות](https://www.tidymodels.org/packages/) עבור מודלים ולמידת מכונה.

- `janitor`: [חבילת janitor](https://github.com/sfirke/janitor) מספקת כלים פשוטים לבחינה וניקוי של נתונים מלוכלכים.

- `corrplot`: [חבילת corrplot](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) מספקת כלי ויזואלי לחקר מטריצת מתאם, שתומך בסידור אוטומטי של משתנים כדי לעזור לזהות דפוסים נסתרים בין משתנים.

ניתן להתקין את החבילות הללו באמצעות:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

הסקריפט הבא בודק אם יש לכם את החבילות הנדרשות להשלמת המודול הזה ומתקין אותן עבורכם במקרה שהן חסרות.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

בהמשך נטען את החבילות המדהימות הללו ונעשה אותן זמינות בסשן הנוכחי של R שלנו. (זה רק להמחשה, `pacman::p_load()` כבר עשה זאת בשבילכם)

## 1. קו רגרסיה ליניארי

כפי שלמדתם בשיעור 1, המטרה של תרגיל רגרסיה ליניארית היא להיות מסוגלים לשרטט *קו* *התאמה* *הטוב ביותר* כדי:

-   **להציג קשרים בין משתנים**. להראות את הקשר בין המשתנים.

-   **לבצע תחזיות**. לבצע תחזיות מדויקות לגבי היכן נקודת נתונים חדשה תיפול ביחס לקו הזה.

כדי לשרטט סוג כזה של קו, אנו משתמשים בטכניקה סטטיסטית שנקראת **רגרסיית ריבועים קטנים**. המונח `ריבועים קטנים` מתייחס לכך שכל נקודות הנתונים שמסביב לקו הרגרסיה מרובעות ואז מסוכמות. באופן אידיאלי, הסכום הסופי יהיה קטן ככל האפשר, מכיוון שאנו רוצים מספר נמוך של שגיאות, או `ריבועים קטנים`. לכן, קו ההתאמה הטוב ביותר הוא הקו שנותן לנו את הערך הנמוך ביותר עבור סכום השגיאות המרובעות - ומכאן השם *רגרסיית ריבועים קטנים*.

אנו עושים זאת מכיוון שאנו רוצים לדגם קו שיש לו את המרחק המצטבר הקטן ביותר מכל נקודות הנתונים שלנו. אנו גם מרבעים את המונחים לפני הסכימה מכיוון שאנו מתמקדים בגודל שלהם ולא בכיוונם.

> **🧮 תראו לי את המתמטיקה**
>
> קו זה, שנקרא *קו ההתאמה הטוב ביותר*, ניתן לביטוי באמצעות [משוואה](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` הוא '`משתנה מסביר` או `מנבא`'. `Y` הוא '`משתנה תלוי` או `תוצאה`'. השיפוע של הקו הוא `b` ו-`a` הוא נקודת החיתוך עם ציר ה-Y, שמתייחסת לערך של `Y` כאשר `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "שיפוע = $y/x$")
    אינפוגרפיקה מאת ג'ן לופר
>
> ראשית, חשבו את השיפוע `b`.
>
> במילים אחרות, ובהתייחס לשאלת הנתונים המקורית שלנו על דלעות: "לחזות את המחיר של דלעת לפי חודש", `X` יתייחס למחיר ו-`Y` יתייחס לחודש המכירה.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    אינפוגרפיקה מאת ג'ן לופר
> 
> חשבו את הערך של Y. אם אתם משלמים בסביבות \$4, כנראה שזה אפריל!
>
> המתמטיקה שמחשבת את הקו חייבת להדגים את השיפוע של הקו, שהוא גם תלוי בנקודת החיתוך, או היכן ש-`Y` ממוקם כאשר `X = 0`.
>
> אתם יכולים לצפות בשיטת החישוב לערכים אלו באתר [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). כמו כן, בקרו ב-[מחשבון ריבועים קטנים](https://www.mathsisfun.com/data/least-squares-calculator.html) כדי לראות כיצד ערכי המספרים משפיעים על הקו.

לא כל כך מפחיד, נכון? 🤓

#### מתאם

מונח נוסף שחשוב להבין הוא **מקדם המתאם** בין משתני X ו-Y נתונים. באמצעות תרשים פיזור, ניתן לראות במהירות את מקדם המתאם הזה. תרשים עם נקודות נתונים מפוזרות בקו מסודר מצביע על מתאם גבוה, אך תרשים עם נקודות נתונים מפוזרות בכל מקום בין X ל-Y מצביע על מתאם נמוך.

מודל רגרסיה ליניארי טוב יהיה כזה שיש לו מקדם מתאם גבוה (קרוב ל-1 יותר מאשר ל-0) באמצעות שיטת רגרסיית ריבועים קטנים עם קו רגרסיה.


## **2. ריקוד עם נתונים: יצירת מסגרת נתונים לשימוש במידול**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>יצירה מאת @allison_horst</figcaption>


<!--![יצירה מאת \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


טען את הספריות והמערכת הנתונים הנדרשות. המרה של הנתונים למסגרת נתונים המכילה תת-קבוצה של הנתונים:

-   קבל רק דלעות שמחירן נקבע לפי יחידת מדידה של "בושל"

-   המרה של התאריך לחודש

-   חישוב המחיר כממוצע של המחירים הגבוהים והנמוכים

-   המרה של המחיר כך שישקף את התמחור לפי כמות בושל

> כיסינו את השלבים הללו ב[שיעור הקודם](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

ברוח ההרפתקה הטהורה, בואו נחקור את [`חבילת השרת`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor) שמספקת פונקציות פשוטות לבחינה וניקוי נתונים מלוכלכים. לדוגמה, בואו נסתכל על שמות העמודות של הנתונים שלנו:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 אנחנו יכולים לעשות טוב יותר. בואו נהפוך את שמות העמודות ל-`friendR` על ידי המרתם לתקן [snake_case](https://en.wikipedia.org/wiki/Snake_case) באמצעות `janitor::clean_names`. כדי לגלות עוד על הפונקציה הזו: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

הרבה tidyR 🧹! עכשיו, ריקוד עם הנתונים באמצעות `dplyr` כמו בשיעור הקודם! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

עבודה טובה!👌 עכשיו יש לך מערך נתונים נקי ומסודר שעליו תוכל לבנות את מודל הרגרסיה החדש שלך!

מה דעתך על גרף פיזור?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


פיזור הנתונים מזכיר לנו שיש לנו נתוני חודשים רק מאוגוסט ועד דצמבר. כנראה שנצטרך יותר נתונים כדי שנוכל להסיק מסקנות בצורה ליניארית.

בואו נבחן שוב את נתוני המודל שלנו:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

מה אם היינו רוצים לחזות את `price` של דלעת בהתבסס על העמודות `city` או `package`, שהן מסוג טקסט? או אפילו בצורה פשוטה יותר, איך נוכל למצוא את הקורלציה (שדורשת ששני הקלטים שלה יהיו מספריים) בין, למשל, `package` ו-`price`? 🤷🤷

מודלים של למידת מכונה עובדים בצורה הטובה ביותר עם מאפיינים מספריים ולא עם ערכי טקסט, ולכן בדרך כלל יש צורך להמיר מאפיינים קטגוריים לייצוגים מספריים.

זה אומר שעלינו למצוא דרך לעצב מחדש את המנבאים שלנו כדי להפוך אותם לקלים יותר לשימוש עבור מודל, תהליך שמכונה `feature engineering`.


## 3. עיבוד מקדים של נתונים למידול עם מתכונים 👩‍🍳👨‍🍳

פעילויות שמעצבות מחדש ערכי מנבאים כדי להפוך אותם לקלים יותר לשימוש עבור מודל נקראות `הנדסת מאפיינים`.

למודלים שונים יש דרישות עיבוד מקדים שונות. לדוגמה, ריבועי מינימום דורשים `קידוד משתנים קטגוריים` כמו חודש, סוג ועיר. זה פשוט כרוך ב`תרגום` עמודה עם `ערכים קטגוריים` לעמודה אחת או יותר `מספריות` שמחליפות את המקור.

לדוגמה, נניח שהנתונים שלך כוללים את המאפיין הקטגורי הבא:

|  עיר   |
|:-------:|
| דנוור  |
| ניירובי |
|  טוקיו  |

ניתן ליישם *קידוד סדרתי* כדי להחליף כל קטגוריה בערך מספרי ייחודי, כך:

| עיר |
|:----:|
|  0   |
|  1   |
|  2   |

וזה בדיוק מה שנעשה לנתונים שלנו!

בקטע הזה, נחקור חבילה מדהימה נוספת של Tidymodels: [recipes](https://tidymodels.github.io/recipes/) - שנועדה לעזור לך לעבד את הנתונים שלך **לפני** אימון המודל. בליבה, מתכון הוא אובייקט שמגדיר אילו שלבים יש ליישם על מערך נתונים כדי להכין אותו למידול.

עכשיו, בואו ניצור מתכון שמכין את הנתונים שלנו למידול על ידי החלפת מספר ייחודי לכל התצפיות בעמודות המנבאים:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

איזה כיף! 👏 הרגע יצרנו את המתכון הראשון שלנו שמגדיר תוצאה (מחיר) ואת המשתנים המנבאים שלה, וכל עמודות המנבאים צריכות להיות מקודדות למספרים שלמים 🙌! בואו נפרק את זה במהירות:

-   הקריאה ל-`recipe()` עם נוסחה אומרת למתכון את *התפקידים* של המשתנים תוך שימוש בנתוני `new_pumpkins` כנקודת ייחוס. לדוגמה, לעמודת `price` הוקצה תפקיד של `outcome`, בעוד שלשאר העמודות הוקצה תפקיד של `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` מציין שכל המנבאים צריכים להיות מומרות למספרים שלמים עם מספור שמתחיל מ-0.

אנחנו בטוחים שיש לכם מחשבות כמו: "זה כל כך מגניב!! אבל מה אם אני צריך לוודא שהמתכונים עושים בדיוק את מה שאני מצפה מהם? 🤔"

זו מחשבה מעולה! תראו, ברגע שהמתכון שלכם מוגדר, אתם יכולים להעריך את הפרמטרים הנדרשים כדי לעבד את הנתונים בפועל, ואז לחלץ את הנתונים המעובדים. בדרך כלל אין צורך לעשות זאת כשמשתמשים ב-Tidymodels (נראה את השיטה המקובלת בעוד רגע-\> `workflows`), אבל זה יכול להיות שימושי כשאתם רוצים לבצע בדיקת תקינות כדי לוודא שהמתכונים עושים את מה שאתם מצפים.

לשם כך, תצטרכו שני פעלים נוספים: `prep()` ו-`bake()`, וכרגיל, חברינו הקטנים מ-R מאת [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) עוזרים לכם להבין את זה טוב יותר!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>יצירה מאת @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): מעריך את הפרמטרים הנדרשים מתוך קבוצת אימון, שניתן ליישם מאוחר יותר על מערכי נתונים אחרים. לדוגמה, עבור עמודת מנבא מסוימת, איזו תצפית תוקצה למספר שלם 0 או 1 או 2 וכו'.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): לוקח מתכון שעבר הכנה ומבצע את הפעולות על כל מערך נתונים.

עם זאת, בואו נכין ונאפה את המתכונים שלנו כדי לאשר באמת שמתחת לפני השטח, עמודות המנבא יעברו קידוד תחילה לפני התאמת המודל.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

יש! 🥳 הנתונים המעובדים `baked_pumpkins` כוללים את כל המשתנים המקודדים, מה שמאשר שהשלבים המקדימים שהגדרנו כמתכון אכן עובדים כמצופה. זה אולי מקשה עליך לקרוא את הנתונים, אבל הופך אותם להרבה יותר מובנים עבור Tidymodels! הקדש זמן לבדוק איזו תצפית הותאמה למספר שלם מתאים.

כדאי גם לציין ש-`baked_pumpkins` הוא מסגרת נתונים (data frame) שניתן לבצע עליה חישובים.

לדוגמה, בואו ננסה למצוא מתאם טוב בין שתי נקודות בנתונים שלך כדי לבנות מודל חיזוי איכותי. נשתמש בפונקציה `cor()` כדי לעשות זאת. הקלד `?cor()` כדי ללמוד עוד על הפונקציה.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


כפי שמתברר, יש רק קשר חלש בין העיר למחיר. עם זאת, יש קשר קצת יותר טוב בין החבילה למחיר שלה. זה הגיוני, נכון? בדרך כלל, ככל שארגז המוצרים גדול יותר, המחיר גבוה יותר.

בזמן שאנחנו עוסקים בזה, בואו גם ננסה להמחיש מטריצת מתאמים של כל העמודות באמצעות חבילת `corrplot`.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 הרבה יותר טוב.

שאלה טובה לשאול עכשיו מהנתונים היא: '`איזה מחיר אני יכול לצפות עבור חבילת דלעת מסוימת?`' בואו נצלול ישר לתוך זה!

> הערה: כאשר אתם **`bake()`** את המתכון המוכן **`pumpkins_prep`** עם **`new_data = NULL`**, אתם מחלצים את נתוני האימון המעובדים (כלומר, מקודדים). אם היה לכם סט נתונים נוסף, לדוגמה סט בדיקה, ורציתם לראות איך המתכון יעבד אותו, פשוט תאפו את **`pumpkins_prep`** עם **`new_data = test_set`**

## 4. בניית מודל רגרסיה ליניארית

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>אינפוגרפיקה מאת דאסאני מדיפאלי</figcaption>


עכשיו, לאחר שבנינו מתכון ואישרנו שהנתונים יעובדו מראש בצורה המתאימה, בואו נבנה מודל רגרסיה כדי לענות על השאלה: `איזה מחיר אני יכול לצפות עבור חבילת דלעת נתונה?`

#### אימון מודל רגרסיה לינארית באמצעות קבוצת האימון

כפי שכנראה כבר הבנתם, העמודה *price* היא משתנה ה`תוצאה`, בעוד שהעמודה *package* היא משתנה ה`מנבא`.

כדי לעשות זאת, תחילה נחלק את הנתונים כך ש-80% ייכנסו לקבוצת האימון ו-20% לקבוצת הבדיקה, לאחר מכן נגדיר מתכון שיקודד את עמודת המנבא למערך של מספרים שלמים, ואז נבנה מפרט מודל. לא נכין ונאפה את המתכון שלנו מכיוון שכבר ידוע לנו שהוא יעבד את הנתונים כפי שצפוי.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

עבודה טובה! עכשיו כשיש לנו מתכון ומפרט מודל, אנחנו צריכים למצוא דרך לאגד אותם יחד לאובייקט שיבצע תחילה עיבוד מקדים לנתונים (prep+bake מאחורי הקלעים), יתאים את המודל לנתונים שעברו עיבוד מקדים וגם יאפשר פעילויות עיבוד לאחרי. איך זה בשביל השקט הנפשי שלך!🤩

ב-Tidymodels, האובייקט הנוח הזה נקרא [`workflow`](https://workflows.tidymodels.org/) והוא מחזיק בצורה נוחה את רכיבי המידול שלך! זה מה שהיינו קוראים לו *pipelines* ב-*Python*.

אז בואו נאגד הכל לתוך workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

בנוסף לכך, ניתן להתאים/לאמן תהליך עבודה בצורה דומה מאוד לאופן שבו ניתן לאמן מודל.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

מהפלט של המודל, ניתן לראות את המקדמים שנלמדו במהלך האימון. הם מייצגים את המקדמים של קו ההתאמה הטוב ביותר שמספק את השגיאה הכוללת הנמוכה ביותר בין המשתנה האמיתי למשתנה החזוי.

#### הערכת ביצועי המודל באמצעות קבוצת הבדיקה

הגיע הזמן לבדוק איך המודל ביצע 📏! איך עושים את זה?

עכשיו, לאחר שאימנו את המודל, נוכל להשתמש בו כדי לבצע תחזיות עבור קבוצת הבדיקה באמצעות `parsnip::predict()`. לאחר מכן, נוכל להשוות את התחזיות הללו לערכי התוויות האמיתיים כדי להעריך עד כמה (או לא!) המודל עובד.

בואו נתחיל עם ביצוע תחזיות עבור קבוצת הבדיקה ואז נחבר את העמודות לקבוצת הבדיקה.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

כן, הרגע אימנתם מודל והשתמשתם בו כדי לבצע תחזיות!🔮 האם הוא טוב? בואו נבחן את ביצועי המודל!

ב-Tidymodels, אנחנו עושים זאת באמצעות `yardstick::metrics()`! עבור רגרסיה לינארית, נתמקד במדדים הבאים:

-   `Root Mean Square Error (RMSE)`: השורש הריבועי של [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). מדד זה מספק ערך מוחלט באותה יחידת מידה כמו התווית (במקרה הזה, המחיר של דלעת). ככל שהערך קטן יותר, כך המודל טוב יותר (במובן הפשוט, זה מייצג את המחיר הממוצע שבו התחזיות שגויות!).

-   `Coefficient of Determination (בדרך כלל נקרא R-squared או R2)`: מדד יחסי שבו ככל שהערך גבוה יותר, כך ההתאמה של המודל טובה יותר. למעשה, מדד זה מייצג כמה מהשונות בין הערכים החזויים לערכים האמיתיים המודל מצליח להסביר.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

הנה ביצועי המודל. בואו נראה אם נוכל לקבל אינדיקציה טובה יותר על ידי הצגת גרף פיזור של החבילה והמחיר, ואז להשתמש בתחזיות שנעשו כדי להוסיף קו מגמה מתאים.

זה אומר שנצטרך להכין ולעבד את קבוצת הבדיקה כדי לקודד את עמודת החבילה ואז לשלב אותה עם התחזיות שנעשו על ידי המודל שלנו.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


מעולה! כפי שניתן לראות, מודל הרגרסיה הליניארית לא מצליח להכליל בצורה טובה את הקשר בין חבילה למחיר המתאים לה.

🎃 מזל טוב, הרגע יצרת מודל שיכול לעזור לחזות את המחיר של כמה סוגים של דלעות. שדה הדלעות שלך לחג יהיה מרהיב. אבל כנראה שתוכל ליצור מודל טוב יותר!

## 5. בניית מודל רגרסיה פולינומי

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>אינפוגרפיקה מאת דאסאני מדיפאלי</figcaption>


<!--![אינפוגרפיקה מאת דאסאני מדיפאלי](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


לפעמים הנתונים שלנו עשויים שלא להיות בעלי קשר ליניארי, אך עדיין נרצה לחזות תוצאה. רגרסיה פולינומית יכולה לעזור לנו לבצע תחזיות עבור קשרים מורכבים יותר שאינם ליניאריים.

לדוגמה, הקשר בין האריזה למחיר במאגר הנתונים של הדלעות שלנו. בעוד שלפעמים יש קשר ליניארי בין משתנים - ככל שהדלעת גדולה יותר בנפח, כך המחיר גבוה יותר - לעיתים קשרים אלו אינם יכולים להיות מוצגים כמישור או כקו ישר.

> ✅ הנה [עוד כמה דוגמאות](https://online.stat.psu.edu/stat501/lesson/9/9.8) לנתונים שיכולים להשתמש ברגרסיה פולינומית  
>
> הסתכלו שוב על הקשר בין הזן למחיר בגרף הקודם. האם פיזור הנתונים הזה נראה כאילו הוא חייב להיות מנותח באמצעות קו ישר? אולי לא. במקרה כזה, תוכלו לנסות רגרסיה פולינומית.  
>
> ✅ פולינומים הם ביטויים מתמטיים שעשויים לכלול משתנה אחד או יותר ומקדמים  

#### אימון מודל רגרסיה פולינומית באמצעות קבוצת האימון  

רגרסיה פולינומית יוצרת *קו מעוקל* שמתאים טוב יותר לנתונים שאינם ליניאריים.

בואו נראה האם מודל פולינומי יבצע תחזיות טובות יותר. נעקוב אחר תהליך דומה לזה שעשינו קודם:

-   יצירת מתכון שמפרט את שלבי העיבוד המקדימים שיש לבצע על הנתונים שלנו כדי להכין אותם למידול, כלומר: קידוד משתנים וחישוב פולינומים בדרגה *n*  

-   בניית מפרט מודל  

-   שילוב המתכון ומפרט המודל לתוך זרימת עבודה  

-   יצירת מודל על ידי התאמת זרימת העבודה  

-   הערכת ביצועי המודל על נתוני הבדיקה  

בואו נתחיל!  


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### הערכת ביצועי המודל

👏👏 יצרת מודל פולינומי, בוא נעשה תחזיות על קבוצת הבדיקה!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

בואו נבדוק איך המודל ביצע על קבוצת הבדיקה באמצעות `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 ביצועים הרבה יותר טובים.

ה-`rmse` ירד מכ-7 לכ-3, מה שמעיד על ירידה בשגיאה בין המחיר האמיתי למחיר החזוי. אפשר *בערך* לפרש זאת כמשמעות שהתחזיות השגויות שגויות בממוצע בכ-3 דולר. ה-`rsq` עלה מכ-0.4 לכ-0.8.

כל המדדים הללו מצביעים על כך שהמודל הפולינומי מתפקד הרבה יותר טוב מהמודל הליניארי. עבודה טובה!

בואו נראה אם נוכל להמחיש את זה!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


אתם יכולים לראות קו מעוקל שמתאים לנתונים שלכם בצורה טובה יותר! 🤩

אתם יכולים להפוך את זה לחלק יותר על ידי העברת נוסחה פולינומית ל-`geom_smooth` כך:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

ממש כמו עקומה חלקה! 🤩

כך תוכל לבצע תחזית חדשה:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


התחזית של `polynomial model` בהחלט הגיונית, בהתחשב בגרפי הפיזור של `price` ו-`package`! ואם זהו מודל טוב יותר מהקודם, בהתבסס על אותם נתונים, תצטרך לתכנן תקציב לדלעות היקרות יותר הללו!

🏆 כל הכבוד! יצרת שני מודלים רגרסיה בשיעור אחד. בחלק האחרון על רגרסיה, תלמד על רגרסיה לוגיסטית כדי לקבוע קטגוריות.

## **🚀אתגר**

בדוק מספר משתנים שונים במחברת זו כדי לראות כיצד הקורלציה משפיעה על דיוק המודל.

## [**שאלון לאחר השיעור**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **סקירה ולימוד עצמי**

בשיעור זה למדנו על רגרסיה ליניארית. ישנם סוגים חשובים נוספים של רגרסיה. קרא על טכניקות Stepwise, Ridge, Lasso ו-Elasticnet. קורס טוב ללמוד ממנו הוא [קורס הלמידה הסטטיסטית של סטנפורד](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

אם אתה רוצה ללמוד יותר על השימוש במסגרת המדהימה של Tidymodels, אנא בדוק את המשאבים הבאים:

-   אתר Tidymodels: [התחל עם Tidymodels](https://www.tidymodels.org/start/)

-   מקס קון וג'וליה סילג, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **תודה ל:**

[אליסון הורסט](https://twitter.com/allison_horst?lang=en) על יצירת האיורים המדהימים שהופכים את R למזמינה ומרתקת יותר. מצא איורים נוספים בגלריה שלה [כאן](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
